<a href="https://colab.research.google.com/github/LatiefDataVisionary/data-science-modelling-and-simulation-college-task/blob/main/fuzzy_mamdani_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. Impor Pustaka dan Definisi Universe**

Langkah pertama adalah mengimpor pustaka yang diperlukan dan mendefinisikan rentang nilai (*universe of discourse*) untuk setiap variabel.


In [5]:
pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 9.8 MB/s eta 0:00:00


In [6]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

In [8]:
# Mendefinisikan rentang nilai (Universe) untuk setiap variabel
# Input 1: Intensitas Cahaya (0-1000 Lux)
cahaya = ctrl.Antecedent(np.arange(0, 1001, 1), 'cahaya')

# Input 2: Jumlah Kendaraan (0-60 kendaraan/menit)
kendaraan = ctrl.Antecedent(np.arange(0, 61, 1), 'kendaraan')

# Output: Daya Lampu (0-100 %)
daya = ctrl.Consequent(np.arange(0, 101, 1), 'daya')